### 숫자 추출 함수
-> 구매수 or 조회 수 구할 때 사용

In [15]:
import pandas as pd
import os
import re

# 문자열에서 숫자 추출하는 함수
def extract_number(value):
    match = re.search(r'(\d+\.?\d*)', str(value))
    number = float(match.group(1)) if match else 0
    return number

# 만, 천 숫자로 변환하는 함수
def convert_to_int(value):
    value = str(value)
    number = extract_number(value)

    if '만' in value:
        return round(number * 10000)
    elif '천' in value:
        return round(number * 1000)
    
    return round(number)

### 행 처리 함수(구매수 or 조회수, 연령, 성별)
-> 행에 있는 데이터들을 구매수, 연령, 성별 데이터를 추출하여 컬럼에 저장

In [16]:
# 구매현황 처리
def process_purchase_row(row):
    if pd.isna(row):
        print("구매현황 데이터가 NaN입니다.")
        return pd.Series({'구매수': 0, '구매_~18세': 0, '구매_19~23세': 0, '구매_24~28세': 0,
                           '구매_29~33세': 0, '구매_34~39세': 0, '구매_40세~': 0, '구매_남성': 0, '구매_여성': 0})

    if not isinstance(row, str):
        print(f"Unexpected data type in 구매현황: {type(row)}")
        return pd.Series({'구매수': 0, '구매_~18세': 0, '구매_19~23세': 0, '구매_24~28세': 0,
                           '구매_29~33세': 0, '구매_34~39세': 0, '구매_40세~': 0, '구매_남성': 0, '구매_여성': 0})
    
    data = row.split('\n')
    
    try:
        purchase_count = convert_to_int(data[0].split(' ')[2])
    except (IndexError, ValueError):
        print(f"구매현황 데이터 형식이 잘못되었습니다: {row}")
        return pd.Series({'구매수': 0, '구매_~18세': 0, '구매_19~23세': 0, '구매_24~28세': 0,
                           '구매_29~33세': 0, '구매_34~39세': 0, '구매_40세~': 0, '구매_남성': 0, '구매_여성': 0})
    
    result = {'구매수': purchase_count}
    
    age_groups = ['~18세', '19~23세', '24~28세', '29~33세', '34~39세', '40세~']
    genders = ['남성', '여성']
    
    # 연령 데이터 처리
    try:
        age_start = data.index('연령') + 8
        age_data = data[age_start:age_start + 12:2]
        for i, age in enumerate(age_groups):
            result[f'구매_{age}'] = round(extract_number(age_data[i]) * purchase_count / 100)
    except (ValueError, IndexError):
        print("연령 데이터를 찾거나 처리하는 데 오류가 발생했습니다.")
        for age in age_groups:
            result[f'구매_{age}'] = 0
    
    # 성별 데이터 처리
    try:
        gender_start = data.index('성별') + 4
        gender_data = data[gender_start:gender_start + 3:2]
        for i, gender in enumerate(genders):
            percentage = extract_number(gender_data[i].split()[-1])
            result[f'구매_{gender}'] = round(percentage * purchase_count / 100)
    except (ValueError, IndexError):
        print("성별 데이터를 찾거나 처리하는 데 오류가 발생했습니다.")
        for gender in genders:
            result[f'구매_{gender}'] = 0
    
    return pd.Series(result)

# 조회현황 처리
def process_view_row(row):
    if pd.isna(row):
        print("조회현황 데이터가 NaN입니다.")
        return pd.Series({'조회수': 0, '조회_~18세': 0, '조회_19~23세': 0, '조회_24~28세': 0,
                           '조회_29~33세': 0, '조회_34~39세': 0, '조회_40세~': 0, '조회_남성': 0, '조회_여성': 0})

    if not isinstance(row, str):
        print(f"Unexpected data type in 조회현황: {type(row)}")
        return pd.Series({'조회수': 0, '조회_~18세': 0, '조회_19~23세': 0, '조회_24~28세': 0,
                           '조회_29~33세': 0, '조회_34~39세': 0, '조회_40세~': 0, '조회_남성': 0, '조회_여성': 0})
    
    data = row.split('\n')
    
    try:
        view_count = convert_to_int(data[0].split(' ')[2])
    except (IndexError, ValueError):
        print(f"조회현황 데이터 형식이 잘못되었습니다: {row}")
        return pd.Series({'조회수': 0, '조회_~18세': 0, '조회_19~23세': 0, '조회_24~28세': 0,
                           '조회_29~33세': 0, '조회_34~39세': 0, '조회_40세~': 0, '조회_남성': 0, '조회_여성': 0})
    
    result = {'조회수': view_count}
    
    age_groups = ['~18세', '19~23세', '24~28세', '29~33세', '34~39세', '40세~']
    genders = ['남성', '여성']
    
    # 연령 데이터 처리
    try:
        age_start = data.index('연령') + 8
        age_data = data[age_start:age_start + 12:2]
        for i, age in enumerate(age_groups):
            result[f'조회_{age}'] = round(extract_number(age_data[i]) * view_count / 100)
    except (ValueError, IndexError):
        print("연령 데이터를 찾거나 처리하는 데 오류가 발생했습니다.")
        for age in age_groups:
            result[f'조회_{age}'] = 0
    
    # 성별 데이터 처리
    try:
        gender_start = data.index('성별') + 4
        gender_data = data[gender_start:gender_start + 3:2]
        for i, gender in enumerate(genders):
            percentage = extract_number(gender_data[i].split()[-1])
            result[f'조회_{gender}'] = round(percentage * view_count / 100)
    except (ValueError, IndexError):
        print("성별 데이터를 찾거나 처리하는 데 오류가 발생했습니다.")
        for gender in genders:
            result[f'조회_{gender}'] = 0
    
    return pd.Series(result)

### 가격 처리 함수

In [17]:
def clean_price(price):
    cleaned_price = price.split('~')[-1].strip().replace(',', '').replace('원', '')
    return round(float(cleaned_price))

### 가격대 구분 함수

In [18]:
# 가격대 구분 함수 정의
def categorize_price(price):
    if price >= 10000 and price < 20000:
        return '1~2만원대'
    elif price >= 30000 and price < 40000:
        return '3~4만원대'
    elif price >= 50000:
        return '5만원대 이상'
    else:
        return '기타'

### 카테고리 분리코드
-> 상위 / 하위카테고리 분리

In [19]:
import pandas as pd

def split_category(category):
    # ' > ' 기준으로 카테고리를 나누기
    category_parts = category.split(' > ')
    
    # 분리된 카테고리 리스트가 3개인 경우
    if len(category_parts) == 3:
        # ' (브랜드명)' 부분을 제거하고 분리
        lower_parts = category_parts[2].split(' (')
        upper_category = category_parts[1].strip()
        sub_category = lower_parts[0].strip()
        brand_name = lower_parts[1][:-1].strip() if len(lower_parts) > 1 else None
        
        # 분리된 값을 반환
        return pd.Series([sub_category, '', brand_name])
    
    # 분리된 카테고리 리스트가 2개인 경우
    elif len(category_parts) == 2:
        # ' (브랜드명)' 부분을 제거하고 분리
        lower_parts = category_parts[1].split(' (')
        sub_category = lower_parts[0].strip()
        brand_name = lower_parts[1][:-1].strip() if len(lower_parts) > 1 else None
        
        # 분리된 값을 반환
        return pd.Series([category_parts[0].strip(), sub_category, brand_name])
    
    # 분리된 카테고리 리스트가 1개 이하인 경우
    else:
        return pd.Series([None, None, None])

### 리뷰 태그

In [20]:
import pandas as pd
import ast

def split_review_tags(df):
    # 태그 카테고리 정의
    tag_categories = ['사이즈', '밝기', '색감', '두께감', '보온성', '무게감', '배송', '포장']
    
    # 카테고리별 열 이름을 위한 리스트
    tag_columns = [f'리뷰태그_{category}' for category in tag_categories]
    
    # 결과를 담을 딕셔너리
    def process_tags(tags):
        # 모든 카테고리에 대해 초기화
        result = {f'리뷰태그_{category}': None for category in tag_categories}
        
        if pd.isna(tags):  # NaN 처리
            return pd.Series(result)
        
        # 문자열 리스트를 실제 리스트로 변환
        try:
            tags_list = ast.literal_eval(tags)
        except (ValueError, SyntaxError):
            return pd.Series(result)
        
        # 각 태그를 카테고리별로 나누어 처리
        for tag in tags_list:
            for category in tag_categories:
                if tag.startswith(category):
                    if '보통이에요' in tag:
                        result[f'리뷰태그_{category}'] = f'{category} 보통이에요'
                    else:
                        if result[f'리뷰태그_{category}'] is None:
                            result[f'리뷰태그_{category}'] = tag
                    break
        
        return pd.Series(result)
    
    # '리뷰태그' 열을 카테고리별로 처리하고 새로운 열로 추가
    review_tags_df = df['리뷰태그'].apply(process_tags)
    
    # 원본 데이터프레임에 병합
    df = pd.concat([df, review_tags_df], axis=1)
    
    # 기존 '리뷰태그' 열 삭제
    df.drop(columns=['리뷰태그'], inplace=True)
    
    return df

### 상품 ID & 리뷰 ID 추가 코드

In [21]:
import pandas as pd

def plus_product_ID(df):
    # 상품명별로 그룹화하고 각 그룹에 대해 고유한 ID 생성
    df['Product ID'] = df.groupby('상품명').ngroup().astype(str).str.zfill(5)
    
    # 'product_' 접두사 추가
    df['Product ID'] = 'product_' + df['Product ID']

def plus_review_ID(df):
    # 상품명별로 그룹화하고 각 그룹에 대해 고유한 ID 생성
    df['Review ID'] = df.groupby('리뷰').ngroup().astype(str).str.zfill(5)
    
    # 'product_' 접두사 추가
    df['Review ID'] = 'review_' + df['Review ID']

### 실행 함수

In [22]:
# 파일 경로 정의
file_path = 'musinsa_crawling_dataset/musinsa_crawling_concat_dataset.csv'

# CSV 파일 읽기
df = pd.read_csv(file_path)

# 데이터 전처리
df['상품좋아요수'] = df['상품좋아요수'].str.replace(',', '', regex=False).astype('int64')
df.drop(columns=['배송'], inplace=True)
df['작성일'] = pd.to_datetime(df['작성일'], errors='coerce').dt.strftime('%Y%m%d').astype('int64')
df['성별'] = df['성별'].map({'남성': 1, '여성': 0})

# 구매현황 및 조회현황 컬럼 처리
purchase_columns = df['구매현황'].apply(process_purchase_row)
view_columns = df['조회현황'].apply(process_view_row)

df = pd.concat([df, purchase_columns, view_columns], axis=1)
df.drop(columns=['구매현황', '조회현황'], inplace=True)

# 가격 데이터 정리
df['상품가격'] = df['상품가격'].apply(clean_price).astype('int64')

# 가격대 컬럼 추가
df['가격대'] = df['상품가격'].apply(categorize_price)

# 카테고리 분리
df[['상위 카테고리', '하위 카테고리', '브랜드명']] = df['카테고리'].apply(split_category)

# 리뷰태그 분리
df = split_review_tags(df)

# Product ID 추가
plus_product_ID(df)

# Review ID 추가
plus_review_ID(df)

# 변환된 데이터를 새로운 CSV 파일로 저장
output_dir = 'musinsa_crawling_dataset'
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, 'musinsa_crawling_columns_preprocessing_dataset.csv')
df.to_csv(output_path, index=False, encoding='utf-8')

print(f"변환된 데이터가 {output_path} 파일로 저장되었습니다.")

구매현황 데이터가 NaN입니다.
구매현황 데이터가 NaN입니다.
구매현황 데이터가 NaN입니다.
구매현황 데이터가 NaN입니다.
구매현황 데이터가 NaN입니다.
구매현황 데이터가 NaN입니다.
구매현황 데이터가 NaN입니다.
구매현황 데이터가 NaN입니다.
구매현황 데이터가 NaN입니다.
구매현황 데이터가 NaN입니다.
조회현황 데이터가 NaN입니다.
조회현황 데이터가 NaN입니다.
조회현황 데이터가 NaN입니다.
조회현황 데이터가 NaN입니다.
조회현황 데이터가 NaN입니다.
조회현황 데이터가 NaN입니다.
조회현황 데이터가 NaN입니다.
조회현황 데이터가 NaN입니다.
조회현황 데이터가 NaN입니다.
조회현황 데이터가 NaN입니다.
변환된 데이터가 musinsa_crawling_dataset\musinsa_crawling_columns_preprocessing_dataset.csv 파일로 저장되었습니다.


### 컬럼이름 영어 변경

In [23]:
# '카테고리' 열 삭제
df.drop(columns=['카테고리'], inplace=True)

# 컬럼 이름 변경
df.rename(columns={
    '상품명': 'product',
    '상품좋아요수': 'product_like',
    '상품가격': 'product_price',
    '가격대': 'price_range',
    '상품태그': 'product_tag',
    '리뷰': 'review',
    '작성일': 'date',
    '평점': 'grade',
    '사이즈': 'size',
    '성별': 'sex',
    '키': 'height',
    '몸무게': 'weight',
    '누적판매량': 'cumulative_sales_volume',
    '도움돼요': 'helpful',
    '스타일 좋아요': 'style_like',
    '구매수': 'purchases',
    '구매_~18세': 'purchases_18',
    '구매_19~23세': 'purchases_19_23',
    '구매_24~28세': 'purchases_24_28',
    '구매_29~33세': 'purchases_29_33',
    '구매_34~39세': 'purchases_34_39',
    '구매_40세~': 'purchases_40',
    '구매_남성': 'purchases_male',
    '구매_여성': 'purchases_female',
    '조회수': 'views',
    '조회_~18세': 'views_18',
    '조회_19~23세': 'views_19_23',
    '조회_24~28세': 'views_24_28',
    '조회_29~33세': 'views_29_33',
    '조회_34~39세': 'views_34_39',
    '조회_40세~': 'views_40',
    '조회_남성': 'views_male',
    '조회_여성': 'views_female',
    '상위 카테고리': 'category',
    '하위 카테고리': 'subcategory',
    '브랜드명': 'brand',
    '리뷰태그_사이즈': 'reviewtag_size',
    '리뷰태그_밝기': 'reviewtag_brightness',
    '리뷰태그_색감': 'reviewtag_color',
    '리뷰태그_두께감': 'reviewtag_thickness',
    '리뷰태그_보온성': 'reviewtag_warmth',
    '리뷰태그_무게감': 'reviewtag_weight',
    '리뷰태그_배송': 'reviewtag_delivery',
    '리뷰태그_포장': 'reviewtag_packaging',
    'Product ID': 'product_ID',
    'Review ID': 'review_ID'
}, inplace=True)

# 변환된 데이터를 새로운 CSV 파일로 저장
output_dir = 'musinsa_crawling_dataset'
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, 'musinsa_crawling_columns_preprocessing_dataset.csv')
df.to_csv(output_path, index=False, encoding='utf-8')

print(f"변환된 데이터가 {output_path} 파일로 저장되었습니다.")

변환된 데이터가 musinsa_crawling_dataset\musinsa_crawling_columns_preprocessing_dataset.csv 파일로 저장되었습니다.


### 컬럼 순서 정렬

In [24]:
# 컬럼 순서 재정렬
desired_order = [
    'index', 'brand', 'category', 'subcategory', 'product', 'product_ID', 'product_like', 'product_price', 'price_range',
    'product_tag', 'cumulative_sales_volume', 'purchases', 'purchases_18', 'purchases_19_23', 'purchases_24_28',
    'purchases_29_33', 'purchases_34_39', 'purchases_40', 'purchases_male', 'purchases_female', 'views', 'views_18',
    'views_19_23', 'views_24_28', 'views_29_33', 'views_34_39', 'views_40', 'views_male', 'views_female', 'date',
    'review', 'review_ID', 'grade', 'size', 'sex', 'height', 'weight', 'helpful', 'style_like', 'reviewtag_size',
    'reviewtag_brightness', 'reviewtag_color', 'reviewtag_thickness', 'reviewtag_warmth', 'reviewtag_weight',
    'reviewtag_delivery', 'reviewtag_packaging'
]

df = df[desired_order]

# 데이터프레임을 CSV 파일로 저장
df.to_csv('musinsa_crawling_dataset\musinsa_crawling_columns_preprocessing_dataset_f.csv', index=False, encoding='utf-8')

### null 값 -> fillna

In [25]:
import pandas as pd

# CSV 파일 읽기
df = pd.read_csv('musinsa_crawling_dataset/musinsa_crawling_columns_preprocessing_dataset_f.csv')

# 'subcategory' 컬럼의 null 값을 '아울렛상품'으로 대체
df['subcategory'].fillna('아울렛상품', inplace=True)

# 'product_tag' 컬럼의 null 값을 '태그없음'으로 대체
df['product_tag'].fillna('태그없음', inplace=True)

# 'reviewtag' 컬럼의 null 값을 '태그없음'으로 대체
df['reviewtag_size'].fillna('태그없음', inplace=True)
df['reviewtag_brightness'].fillna('태그없음', inplace=True)
df['reviewtag_color'].fillna('태그없음', inplace=True)
df['reviewtag_thickness'].fillna('태그없음', inplace=True)
df['reviewtag_warmth'].fillna('태그없음', inplace=True)
df['reviewtag_weight'].fillna('태그없음', inplace=True)
df['reviewtag_delivery'].fillna('태그없음', inplace=True)
df['reviewtag_packaging'].fillna('태그없음', inplace=True)

# 'sex', 'height', 'weight' 컬럼에서 null 값을 가진 행 삭제
df.dropna(subset=['sex', 'height', 'weight'], inplace=True)


# 변경된 데이터프레임을 새로운 CSV 파일로 저장
df.to_csv('musinsa_crawling_dataset/musinsa_crawling_columns_preprocessing_dataset_f.csv', index=False)

# 'sex', 'height', 'weight' 컬럼에서 null 값이 있는 행의 개수 출력 (삭제 후 확인용)
null_sex_count = df['sex'].isnull().sum()
null_height_count = df['height'].isnull().sum()
null_weight_count = df['weight'].isnull().sum()

print(f"Null 값이 있는 'sex' 컬럼의 행 개수: {null_sex_count}")
print(f"Null 값이 있는 'height' 컬럼의 행 개수: {null_height_count}")
print(f"Null 값이 있는 'weight' 컬럼의 행 개수: {null_weight_count}")


C:\Users\USER\AppData\Local\Temp\ipykernel_1156\317276008.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['subcategory'].fillna('아울렛상품', inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_1156\317276008.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

Null 값이 있는 'sex' 컬럼의 행 개수: 0
Null 값이 있는 'height' 컬럼의 행 개수: 0
Null 값이 있는 'weight' 컬럼의 행 개수: 0


### 하위카테고리 sum
1. 상의
   1. 긴소매 티셔츠 -> 긴소매 티셔츠
   2. 맨투맨/스웨트, 니트/스웨트 -> 맨투맨/니트/스웨트
   3. 반소매 티셔츠, 민소매 티셔츠, 피케/카라 티셔츠 -> 민소매/반소매/카라 티셔츠
   4. 셔츠/블라우스 -> 셔츠/블라우스
   5. 아울렛상품 -> 아울렛상품
2. 바지
   1. 기타 바지, 코튼 팬츠, 트래이닝/조거 팬츠 -> 기타 바지
   2. 데님 팬츠 -> 데님 팬츠
   3. 숏 팬츠 -> 숏 팬츠
   4. 슈트 팬츠/슬랙스
   5. 아울렛상품
3. 아우터
   1. 겨울 싱글 코트, 겨울 기타 코트 -> 겨울 코트
   2. 후드 집업 -> 후드 집업
   3. 숏패딩/헤비 아우터 -> 숏패딩/헤비 아우터
   4. 슈트/ 블레이저 재킷
   5. 기타 아우터, 플리스/뽀글이 -> 플리스/뽀글이/기타 아우터
   6. 블루종/ MA-1, 트러커 재킷 -> 블루종/트러커 재킷
   7. 나일론/ 코치 재킷, 아노락 재킷-> 나일론/코치, 아노락 재킷
   8. 베스트, 카디건 -> 베스트/카디건
   9. 아울렛상품 -> 아울렛상품

In [26]:
import pandas as pd

# 하위 카테고리 매핑 정의
subcategory_mapping = {
    '긴소매 티셔츠': '긴소매 티셔츠',
    '맨투맨/ 스웨트': '맨투맨/니트/스웨트',
    '니트/스웨터': '맨투맨/니트/스웨트',
    '반소매 티셔츠': '민소매/반소매/카라 티셔츠',
    '민소매 티셔츠': '민소매/반소매/카라 티셔츠',
    '피케/카라 티셔츠': '민소매/반소매/카라 티셔츠',
    '셔츠/블라우스': '셔츠/블라우스',
    '아울렛상품': '아울렛상품',
    '기타 바지': '기타 바지',
    '코튼 팬츠': '기타 바지',
    '트레이닝/조거 팬츠': '기타 바지',
    '트래이닝/조거 팬츠': '기타 바지',
    '데님 팬츠': '데님 팬츠',
    '숏 팬츠': '숏 팬츠',
    '슈트 팬츠/슬랙스': '슈트 팬츠/슬랙스',
    '겨울 싱글 코트': '겨울 코트',
    '겨울 기타 코트': '겨울 코트',
    '후드 집업': '후드 집업',
    '숏패딩/헤비 아우터': '숏패딩/헤비 아우터',
    '슈트/ 블레이저 재킷': '슈트/ 블레이저 재킷',
    '기타 아우터': '플리스/뽀글이/기타 아우터',
    '플리스/뽀글이': '플리스/뽀글이/기타 아우터',
    '블루종/ MA-1': '블루종/트러커 재킷',
    '트러커 재킷': '블루종/트러커 재킷',
    '나일론/ 코치 재킷': '나일론/코치, 아노락 재킷',
    '아노락 재킷': '나일론/코치, 아노락 재킷',
    '베스트': '베스트/카디건',
    '카디건': '베스트/카디건'
}

# subcategory 컬럼의 값을 매핑
df['subcategory'] = df['subcategory'].map(subcategory_mapping)

# 결과 확인
print(df)

# 결과를 CSV 파일로 저장
df.to_csv('musinsa_crawling_dataset\musinsa_crawling_columns_preprocessing_dataset_f.csv', index=False)

       index  brand category     subcategory  \
0          1  에잇세컨즈       상의  민소매/반소매/카라 티셔츠   
1          2  에잇세컨즈       상의  민소매/반소매/카라 티셔츠   
2          3  에잇세컨즈       상의  민소매/반소매/카라 티셔츠   
3          4  에잇세컨즈       상의  민소매/반소매/카라 티셔츠   
4          5  에잇세컨즈       상의  민소매/반소매/카라 티셔츠   
...      ...    ...      ...             ...   
25880  25881     탑텐      아우터     슈트/ 블레이저 재킷   
25881  25882     탑텐      아우터     슈트/ 블레이저 재킷   
25882  25883     탑텐      아우터     슈트/ 블레이저 재킷   
25883  25884     탑텐      아우터     슈트/ 블레이저 재킷   
25884  25885     탑텐      아우터     슈트/ 블레이저 재킷   

                                         product     product_ID  product_like  \
0              베이직 라운드넥 반소매 티셔츠 화이트 (324242LY11)  product_00071          1845   
1              베이직 라운드넥 반소매 티셔츠 화이트 (324242LY11)  product_00071          1845   
2              베이직 라운드넥 반소매 티셔츠 화이트 (324242LY11)  product_00071          1845   
3              베이직 라운드넥 반소매 티셔츠 화이트 (324242LY11)  product_00071          1845   
4              베이직

### Topic 컬럼 추가 -> 빈상태로 추가 추후 업데이트

In [27]:
# 'topic' 컬럼 추가 (값이 없는 상태로 설정)
df['topic'] = pd.NA

# 변경된 데이터프레임을 새로운 CSV 파일로 저장
df.to_csv('musinsa_crawling_dataset\musinsa_crawling_columns_preprocessing_dataset_f.csv', index=False)

### 데이터프레임 info 확인용

In [29]:
import pandas as pd

df = pd.read_csv('musinsa_crawling_dataset/musinsa_crawling_columns_preprocessing_dataset_f.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25878 entries, 0 to 25877
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   index                    25878 non-null  int64  
 1   brand                    25878 non-null  object 
 2   category                 25878 non-null  object 
 3   subcategory              25878 non-null  object 
 4   product                  25878 non-null  object 
 5   product_ID               25878 non-null  object 
 6   product_like             25878 non-null  int64  
 7   product_price            25878 non-null  int64  
 8   price_range              25878 non-null  object 
 9   product_tag              25878 non-null  object 
 10  cumulative_sales_volume  25878 non-null  object 
 11  purchases                25878 non-null  int64  
 12  purchases_18             25878 non-null  int64  
 13  purchases_19_23          25878 non-null  int64  
 14  purchases_24_28       